## Introduction
![Pikachu](https://img.gifmagazine.net/gifmagazine/images/31051/medium_thumb.png)
**1. Which is the best Generation?**

Pokemon is the japanese icon today. Now there are more than 800 characters and 18 type in Pokemon. Pokemon's status depends on its tribe. For example,　Mewtwo, legendary pokemon has the best status in 1st Generation. Then which generations dose have the best status? 

**2. Correlation of Pokemon's Stats**

Are there any correlation among Pokemon's stats? Check and draw heatmap for finding correlation of Pokemon's stats.

**3. The Relation of Type and Stats**

Pokemons' tyepe is various. We guess that Pokemon whose type is "fighting" is good at "attack" or psychic Pokemon is nice for "special attack". Dose Pokemon's type affect against their stats? 


**Pokemons status' summary**
* **HP**: hit points, or health, defines how much damage a pokemon can withstand before fainting.
* **Attack**: the base modifier for normal attacks.
* **Defense**: the base damage resistance against normal attacks.
* **SP Atk**: special attack, the base modifier for special attacks.
* **SP Def**: the base damage resistance against special attacks.
* **Speed**: determines which pokemon attacks first each round.
******

## Spider Plot Visualization on their Stats
**In the beginning**, status of *Bulbasaur*, *Ivysaur*, *Venusaur* that ia one of the most common Pokemon are as follow.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import pi

df = pd.read_csv("../input/pokemon/Pokemon.csv")
df_status = df[["Name","HP","Attack","Defense","Sp. Atk","Sp. Def","Speed"]]

### spider plot ###
def make_spider(df, nrows, ncols, index, color, label=None, title=None, ax=None):
    categories=list(df)[1:]
    N = len(categories)
    
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]

    if ax == None:
        ax = plt.subplot(nrows, ncols, index+1, polar=True)

    ax.set_theta_offset(pi / 2)
    ax.set_theta_direction(-1)

    plt.xticks(angles[:-1], categories, color='grey', size=8)

    ax.set_rlabel_position(0)
    plt.yticks([40,80,120], ["40","80","120"], color="grey", size=7)
    plt.ylim(0,160)

    values=df.loc[row].drop('Name').values.flatten().tolist()
    values += values[:1]
    ax.plot(angles, values, color=color, linewidth=2, linestyle='solid', label=label)
    ax.fill(angles, values, color=color, alpha=0.4)

    if title:
        plt.title(title, size=11, color=color, y=1.1)
        
my_dpi=60
plt.figure(figsize=(1000/my_dpi, 1000/my_dpi), dpi=my_dpi)

cols = ["green","green","green","green"]

nr, nc = 1,4
for row in range(0,4):
    make_spider(df_status, nr, nc, row, title=df_status['Name'][row], color=cols[row])

plt.show()     

**Spider plot** shows their status and clearly, Pokemons status depends on their level of evolution.

## 1. The Best Generation
Which dose generation have the best status? Violin plot of Pokemon's total stats by generation is as follow. 

In [ ]:
plt.figure(figsize=(13,10), dpi=80)
sns.violinplot(x='Generation', y='Total', data=df, scale='width', inner='quartile')

plt.title('Violin Plot of Total Stats by Generation', fontsize=22)
plt.show()

**Clear Differences** aren't found among generation. Then, is every generation same level on battle? It is too early for decision.

**Speed** is the most important stats for pokemon battle. Because fast pokemon can attack or do action earlier than slow one. So, let's check about speed stats!

In [ ]:
plt.figure(figsize=(13,10), dpi=80)
sns.violinplot(x='Generation', y='Speed', data=df, scale='width', inner='quartile')

plt.title('Violin Plot of Speed Stats by Generation', fontsize=22)
plt.show()

**We** can see that generation 1, 4 and 5 are better than others. howevere, there isn't huge difference, too.

**How about legendary**? Legendary Pokemons have high potential compared to ordinal Pokemons. So, if rate of legendary pokemon of a generation is the highest, that is the best generation. Checking proportion of legendary and drawing barplot.

In [ ]:
#preprocessing
df.loc[df["Legendary"]==False,"Legendaryid"] = 0
df.loc[df["Legendary"]==True,"Legendaryid"] = 1

# calculate proportion of legendary #
legendary_ratio = df.groupby("Generation").mean()["Legendaryid"]
legendary_ratio

In [ ]:
sns.set_style('darkgrid')
df_plot = pd.DataFrame(columns={"Generation","Rate","colors"})
x = legendary_ratio.values
df_plot["Generation"] = legendary_ratio.index
df_plot['Rate'] = (x - x.mean())/x.std()
df_plot['colors'] = ['red' if x < 0 else 'green' for x in df_plot['Rate']]
df_plot.sort_values('Rate', inplace=True)
df_plot.reset_index(inplace=True)

plt.figure(figsize=(14, 10))
plt.hlines(
    y=df_plot.index, xmin=0, xmax=df_plot.Rate,
    color=df_plot.colors,
    alpha=.4,
    linewidth=5)

plt.gca().set(xlabel='Legendary Rate', ylabel='Generation')
plt.yticks(df_plot.index, df_plot.Generation, fontsize=12)
plt.title('Diverging Bars of Legendary Rate', fontdict={'size':20})
plt.show()

**Obviously**, Generation 3 and 4 have more legendary Pokemon than others.



**Conlusion: Which generation is the best?**
1. <font color="Red">Generation 4</font> is the best. Because they have many legendary and they are faster than others
1. Lately generation includes more legendary Pokemon than old generations

## 2. Correlation of Pokemon's stats
**At first**, draw heatmap and heatmap of Pokemon stats is　the diagram below.


In [ ]:
corr = df_status.corr()

### heat map ###
mask = np.triu(np.ones_like(corr, dtype=np.bool))

f, ax = plt.subplots(figsize=(10, 10))

cmap = sns.diverging_palette(255, 133, l=60, n=24, center = "light")

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr,
            mask=mask, cmap=cmap, vmax=.6, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

plt.show()

**Heatmap says** there are good correlation between Sp.Def and Ap.Atk & Sp.Def and Defense. Let's look at the hex plot and check relationship between two.

In [ ]:
p1 = sns.jointplot(x="Sp. Atk",y="Sp. Def",data=df,kind="hex",color="orange")
p1.fig.suptitle("Hex Plot of Sp. Atk and Sp. Def")
p1.fig.subplots_adjust(top=0.95)
p2 = sns.jointplot(x="Defense",y="Sp. Def",data=df,kind="hex",color="green")
p2.fig.suptitle("Hex Plot of Defense and Sp. Def")
p2.fig.subplots_adjust(top=0.95)
p3 = sns.jointplot(x="Defense",y="Speed",data=df,kind="hex",color="red")
p3.fig.suptitle("Hex Plot of Defense and Speed -No Correlation")
p3.fig.subplots_adjust(top=0.95)

**Clearly**, there are high correlation between these sets.



**Conlusion: Correlation of Pokemon Stats**
1. Special defense affects their <font color="Orange">"Special attack"</font> and <font color="Green">"Defense"</font>.
1. Deffence dosen't affect their speed at all.

## 3. The Relation of Type and Stats

In [ ]:
plt.figure(figsize=(13,10), dpi=80)
sns.violinplot(x='Type 1', y='Total', data=df, scale='width', inner='quartile')

plt.title('Violin Plot of HP Stats by Type', fontsize=22)
plt.show()

Their total stats are almost same except Dragon. Dragon has high status compared to others.

In [ ]:
attack_byType = df.groupby("Type 1").mean()["Attack"]

df_plot2 = pd.DataFrame(columns={"Type","Attack","colors"})
x = attack_byType.values
df_plot2["Type"] = attack_byType.index
df_plot2['Attack'] = (x - x.mean())/x.std()
df_plot2['colors'] = ['red' if x < 0 else 'green' for x in df_plot2['Attack']]
df_plot2.sort_values('Attack', inplace=True)

plt.figure(figsize=(14,14), dpi=80)
plt.hlines(y=df_plot2.Type, xmin=0, xmax=df_plot2.Attack)
for x, y, tex in zip(df_plot2.Attack, df_plot2.Type, df_plot2.Attack):
    t = plt.text(
        x, y, round(tex, 2),
        horizontalalignment='right' if x < 0 else 'left',
        verticalalignment='center',
        fontdict={'color':'red' if x < 0 else 'green', 'size':14})

plt.yticks(df_plot2.Type, df_plot2.Type, fontsize=12)
plt.title('Diverging Text Bars of Attack by Type', fontdict={'size':20})
plt.xlim(-3, 3)
plt.show()

**"Dragon"** and "Fighting" is good at "Attack", but "Fairly" is weak.

**Although** thier total stats is almost same regardless of their type, Pokemon's type has huge effection against their specific stats.

## Conclusion
* Defference among Generation
> 1. <font color="Red">Generation 4</font> is the best. Because they have many legendary and they are faster than others
> 1. Lately generation includes more legendary Pokemon than old generations
* Stats correlation
> 1. Special defense affects their <font color="Orange">"Special attack"</font> and <font color="Green">"Defense"</font>.
> 1. Deffence dosen't affect their speed at all.
* Type Effection on Stats
> 1. Thier total stats is almost same regardless of their type except Dragon
> >  <font color="Purple">Dragon</font> has by far the best stats.
> 1. Obviouslyokemon's type has huge effection against their specific stats.
> > For example, <font color="Green">Bug</font> have low HP, <font color="Brown">Fighting</font> have good attack